In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
from langchain.agents import tool

In [119]:
# dummy function and we'll see whats wrong with it

@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

In [120]:
search.name

'search'

In [121]:
search.description

'search(query: str) -> str - Search for weather online'

In [6]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [7]:
# note that parameters do not have description

In [8]:
from pydantic import BaseModel, Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


In [9]:
# fixed description in parameters by passing pdyanditc function schema into decorator

In [10]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

In [11]:
search.args

{'query': {'title': 'Query',
  'description': 'Thing to search for',
  'type': 'string'}}

In [12]:
search.run("sf")

'42f'

In [15]:
search("sf")

'42f'

In [26]:
# actual function returning temperature

In [16]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [17]:
get_current_temperature.name

'get_current_temperature'

In [18]:
get_current_temperature.description

'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.'

In [19]:
get_current_temperature.args

{'latitude': {'title': 'Latitude',
  'description': 'Latitude of the location to fetch weather data for',
  'type': 'number'},
 'longitude': {'title': 'Longitude',
  'description': 'Longitude of the location to fetch weather data for',
  'type': 'number'}}

In [20]:
# creating a function/tool-schema compatible with openai's client

In [21]:
from langchain.tools.render import format_tool_to_openai_function

In [22]:
format_tool_to_openai_function(get_current_temperature)

{'name': 'get_current_temperature',
 'description': 'get_current_temperature(latitude: float, longitude: float) -> dict - Fetch current temperature for given coordinates.',
 'parameters': {'title': 'OpenMeteoInput',
  'type': 'object',
  'properties': {'latitude': {'title': 'Latitude',
    'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'title': 'Longitude',
    'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude']}}

In [27]:
# static testing tool
get_current_temperature({"latitude": 30.72811, "longitude": 76.77065})

'The current temperature is 28.6°C'

In [136]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries[:2])

In [137]:
search_wikipedia.name

'search_wikipedia'

In [138]:
search_wikipedia.description

'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.'

In [139]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'search_wikipedia(query: str) -> str - Run Wikipedia search and get page summaries.',
 'parameters': {'title': 'search_wikipediaSchemaSchema',
  'type': 'object',
  'properties': {'query': {'title': 'Query', 'type': 'string'}},
  'required': ['query']}}

In [140]:
search_wikipedia({"query": "langchain"})

"Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Milvus (vector database)\nSummary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.\nMilvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.\n\n"

In [43]:
# most of the times the tools we create make use of external api calls, 
# OpenAPISpec provides detailed specification of such endpoints
# openapi_spec_to_openai_fn conversts such callable methods to openai compatible functions like done with pydantic conversion

In [44]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [34]:
# OpenAPISpec example
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [35]:
spec = OpenAPISpec.from_text(text)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [36]:
spec

OpenAPISpec(openapi='3.0.0', info=Info(title='Swagger Petstore', summary=None, description=None, termsOfService=None, contact=None, license=License(name='MIT', identifier=None, url=None), version='1.0.0'), jsonSchemaDialect=None, servers=[Server(url='http://petstore.swagger.io/v1', description=None, variables=None)], paths={'/pets': PathItem(ref=None, summary=None, description=None, get=Operation(tags=['pets'], summary='List all pets', description=None, externalDocs=None, operationId='listPets', parameters=[Parameter(name='limit', param_in='query', description='How many items to return at one time (max 100)', required=False, deprecated=False, allowEmptyValue=False, style=None, explode=False, allowReserved=False, param_schema=Schema(allOf=None, anyOf=None, oneOf=None, schema_not=None, schema_if=None, then=None, schema_else=None, dependentSchemas=None, prefixItems=None, items=None, contains=None, properties=None, patternProperties=None, additionalProperties=None, propertyNames=None, unev

In [46]:
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [47]:
pet_openai_functions

[{'name': 'listPets',
  'description': 'List all pets',
  'parameters': {'type': 'object',
   'properties': {'params': {'type': 'object',
     'properties': {'limit': {'type': 'integer',
       'maximum': 100.0,
       'schema_format': 'int32',
       'description': 'How many items to return at one time (max 100)'}},
     'required': []}}}},
 {'name': 'createPets',
  'description': 'Create a pet',
  'parameters': {'type': 'object', 'properties': {}}},
 {'name': 'showPetById',
  'description': 'Info for a specific pet',
  'parameters': {'type': 'object',
   'properties': {'path_params': {'type': 'object',
     'properties': {'petId': {'type': 'string',
       'description': 'The id of the pet to retrieve'}},
     'required': ['petId']}}}}]

In [52]:
# testing model's listed args for tool call

In [53]:
from langchain.chat_models import ChatOpenAI

In [54]:
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

In [55]:
model.invoke("what are three pets names")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'listPets', 'arguments': '{"params":{"limit":3}}'}})

In [56]:
model.invoke("tell me about pet with id 42")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'showPetById', 'arguments': '{"path_params":{"petId":"42"}}'}})

In [57]:
# routing
# exposing LLM with diff tools
# LLM decides which path to take and also the args for that path

In [58]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [59]:
model.invoke("what is the weather in sf right now")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude":37.7749,"longitude":-122.4194}'}})

In [60]:
model.invoke("what is langchain")

AIMessage(content='', additional_kwargs={'function_call': {'name': 'search_wikipedia', 'arguments': '{"query":"Langchain"}'}})

In [90]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "answer to user's query as if you were Tony Stark"),
    ("user", "{input}"),
])
chain = prompt | model

In [91]:
chain.invoke({"input": "what is the weather in sf right now"})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_current_temperature', 'arguments': '{"latitude":37.7749,"longitude":-122.4194}'}})

In [103]:
# OpenAIFunctionsAgentOutputParser
# to return an object which has distinct usable fields
# just converts the object into more readable/usable one (nothing major)

In [104]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [105]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [111]:
result = chain.invoke({"input": "what is the weather in sector 20a, chandigarh right now"})

In [112]:
type(result)

langchain.schema.agent.AgentActionMessageLog

In [113]:
result.tool

'get_current_temperature'

In [114]:
result.tool_input

{'latitude': 30.7347, 'longitude': 76.7936}

In [115]:
get_current_temperature(result.tool_input)

'The current temperature is 28.2°C'

In [116]:
result = chain.invoke({"input": "wassup baby!"})

In [117]:
type(result)

langchain.schema.agent.AgentFinish

In [122]:
# NOTE

# when asked about whether it returned object -> langchain.schema.agent.AgentActionMessageLog
# while when asked a simple query, object returned = langchain.schema.agent.AgentFinish


# its pretty intuitive though:
# when tool call -> AgentAction
# when a simple query -> AgentFinish

In [123]:
result.return_values

{'output': 'Hey there! How can I assist you today?'}

In [124]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [125]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [126]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [127]:
result

'The current temperature is 14.5°C'

In [141]:
result = chain.invoke({"input": "What is langchain?"})

In [142]:
result

"Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: Milvus (vector database)\nSummary: Milvus is a distributed vector database developed by Zilliz. It is available as both open-source software and a cloud service.\nMilvus is an open-source project under LF AI & Data Foundation distributed under the Apache License 2.0.\n\n"

In [130]:
chain.invoke({"input": "hi!"})

'Hello! How can I assist you today?'

In [144]:
# creating a chatbot

In [161]:
import wikipedia
from langchain.tools import tool

@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[:3]:
        try:
            wiki_page = wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            wikipedia.exceptions.PageError,
            wikipedia.exceptions.DisambiguationError,
        ):
            continue  # Skip problematic pages
    if not summaries:
        return "No good Wikipedia search result was found."
    return "\n\n".join(summaries[:2])


In [147]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [148]:
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [164]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "answer to user's query as if you were Tony Stark"),
    ("user", "prompt: {input} and context: {context}"),
    ("system", "remember to give most weightage to last object in context that is context[-1] and ignore other objects if they are unrelated")
])

In [167]:
prompt2 = ChatPromptTemplate.from_messages(
[
    ("system", "answer based on given context conversation of user and ai: {context}"),
])

In [168]:
messages = []

tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }

chain = prompt | model | OpenAIFunctionsAgentOutputParser()
chain2 = prompt2 | model | OpenAIFunctionsAgentOutputParser()
        
while True:
    query = input()
    if query.strip().lower() == "quit":
        break

    messages.append({
        "user": query
    })
    res = chain.invoke({"input": query, "context": messages})
    while not isinstance(res, AgentFinish):
        print(f"calling tool: {res.tool} with args: {res.tool_input}")
        tool_res = tools[res.tool].run(res.tool_input)
        messages.append({
            "ai": tool_res
        })
        res = chain2.invoke({"context": messages})
    print(res.return_values['output'])

hi
Hey there! How can I assist you today?
whats the weather like in chd
calling tool: get_current_temperature with args: {'latitude': 30.7333, 'longitude': 76.7794}
calling tool: get_current_temperature with args: {'latitude': 30.7333, 'longitude': 76.7794}
The current temperature is 28.2°C
do you know anything baout lamine yamal
calling tool: search_wikipedia with args: {'query': 'Lamine Yamal'}
Lamine Yamal Nasraoui Ebana (born 13 July 2007) is a Spanish professional footballer who plays as a winger for La Liga club Barcelona and the Spain national team. Known for his flair, chance creation, and long-distance curling goals, Yamal is regarded as one of the best players in the world.

Yamal was a member of the youth academy, La Masia, before breaking into the Barcelona first team squad in the 2023–24 season. He became the youngest player to be nominated for the Ballon d'Or (at age 17) and also won the Kopa Trophy in 2024, presented to the best young player in the world. In the followin